# Лабораторная работа 1. Pandas, метод ближайших соседей и решающие деревья.

ФИО:

Группа: 

In [1]:
import numpy as np
import pandas as pd

## Часть 1: Pandas

#### Ответьте на вопросы о данных по авиарейсам в США за январь-апрель 2008 года.

Данные: http://stat-computing.org/dataexpo/2009/2008.csv.bz2
(обратите внимание, что распаковывать этот файл не обязательно — функция `pandas.read_csv` умеет читать из архивов автоматически)

Описание: http://stat-computing.org/dataexpo/2009/the-data.html

1. Какая из причин отмены рейса (`CancellationCode`) была самой частой? (расшифровки кодов можно найти в описании данных)
2. Найдите среднее, минимальное и максимальное расстояние, пройденное самолетом.
3. Не выглядит ли подозрительным минимальное пройденное расстояние? В какие дни и на каких рейсах оно было? Какое расстояние было пройдено этими же рейсами в другие дни?
4. Из какого аэропорта было произведено больше всего вылетов? В каком городе он находится?
5. Найдите для каждого аэропорта среднее время полета (`AirTime`) по всем вылетевшим из него рейсам. Какой аэропорт имеет наибольшее значение этого показателя?
6. Найдите аэропорт, у которого наибольшая доля задержанных (`DepDelay > 0`) рейсов. Исключите при этом из рассмотрения аэропорты, из которых было отправлено меньше 1000 рейсов (используйте функцию `filter` после `groupby`).

In [ ]:
data = pd.read_csv('2008.csv')
print(data.shape)
data = data[(data.Month > 0.5) & (data.Month < 4.5)]
print(data.shape)

In [4]:
data.shape

(2389217, 29)

In [11]:
data.CancellationCode.value_counts()

A    26075
B    25744
C    12617
D        6
Name: CancellationCode, dtype: int64

A - carrier. По-видимому, проблемы с перевозчиком.

На втором месте погода.

In [6]:
data.Distance.describe()

count    2389217.000000
mean         726.158911
std          557.176261
min           17.000000
25%          325.000000
50%          583.000000
75%          957.000000
max         4962.000000
Name: Distance, dtype: float64

* Минимальное расстояние - 17 миль
* Среднее - 726.16
* Максимальное - 4962

In [14]:
data.loc[data.Distance == 17, ['Month', 'DayofMonth', 'DayofWeek', 'FlightNum']]

,Month,DayofMonth,DayofWeek,FlightNum
1688702,3,8,NaN,1368


In [15]:
data.loc[data.FlightNum == 1368, ['Distance']].describe()

,Distance
count,224.000000
mean,823.321429
std,271.211903
min,17.000000
25%,950.000000
50%,950.000000
75%,950.000000
max,992.000000


In [37]:
data.loc[(data.FlightNum == 1368) & (data.Distance < 290)]

,Year,Month,DayofMonth,DayOfWeek,DepTime,CRSDepTime,ArrTime,CRSArrTime,UniqueCarrier,FlightNum,...,TaxiIn,TaxiOut,Cancelled,CancellationCode,Diverted,CarrierDelay,WeatherDelay,NASDelay,SecurityDelay,LateAircraftDelay
1688702,2008,3,8,6,NaN,1105,NaN,1128,AA,1368,...,NaN,NaN,1,B,0,NaN,NaN,NaN,NaN,NaN


Больше 75% полетов проходило на расстояние больше 950 миль. Полет на 17 миль был единственным с дальностью полета меньше 290 миль.

In [52]:
data.groupby('Origin').size().argmax()

'ATL'

Город Атланта

In [38]:
data.groupby('Origin')['AirTime'].mean()

Origin
ABE     90.965040
ABI     36.690141
ABQ     91.471470
ABY     34.963415
ACT     30.539860
ACV     53.636610
ACY    108.848214
ADK    149.193548
ADQ     41.964286
AEX     70.034704
AGS     39.355207
AKN     18.500000
ALB    105.005062
ALO     36.679487
AMA     56.313119
ANC    136.479326
ASE     57.575113
ATL     94.947786
ATW     57.029063
AUS     97.524117
AVL     73.666667
AVP     99.535475
AZO     36.399687
BDL    122.260353
BET     56.849398
BFL     64.265053
BGM     67.872247
BGR    105.571965
BHM     84.092114
BIL     70.626725
          ...    
SPI     47.264000
SPS     30.099346
SRQ    109.142414
STL    100.017373
STT    168.322909
STX    177.161491
SUN     47.930337
SUX     45.840000
SWF    140.552967
SYR     85.121257
TEX     92.679389
TLH     65.688283
TOL     56.487666
TPA    120.292551
TRI     48.850153
TUL     83.150345
TUP     42.900000
TUS     95.136052
TVC     47.696909
TWF     42.013774
TXK     47.098734
TYR     31.688380
TYS     83.134138
VLD     45.033233
VPS

In [40]:
data.groupby('Origin')['AirTime'].mean().argmax(), data.groupby('Origin')['AirTime'].mean().max()

('SJU', 209.90795031055902)

In [74]:
data2 = data.groupby('Origin').filter(lambda x: len(x) >= 1000)[['Origin', 'DepDelay']]

In [75]:
size = data2.groupby('Origin').size()
count = data2[data2.DepDelay > 0].groupby('Origin').size()

In [80]:
(count / size).argmax()

'HOU'

In [81]:
(count / size)['HOU']

0.59681640937984082

## Часть 2: метрические методы и категориальные признаки

Все дальнейшие эксперименты предлагается проводить на данных соревнования Amazon Employee Access Challenge: https://www.kaggle.com/c/amazon-employee-access-challenge

В данной задаче предлагается предсказать, будет ли одобрен запрос сотрудника на получение доступа к тому или иному ресурсу. Все признаки являются категориальными.

Для удобства данные можно загрузить по ссылке: https://www.dropbox.com/s/q6fbs1vvhd5kvek/amazon.csv

Сразу прочитаем данные и создадим разбиение на обучение и контроль:

In [2]:
data = pd.read_csv('amazon.csv')
data.head()

,ACTION,RESOURCE,MGR_ID,ROLE_ROLLUP_1,ROLE_ROLLUP_2,ROLE_DEPTNAME,ROLE_TITLE,ROLE_FAMILY_DESC,ROLE_FAMILY,ROLE_CODE
0,1,39353,85475,117961,118300,123472,117905,117906,290919,117908
1,1,17183,1540,117961,118343,123125,118536,118536,308574,118539
2,1,36724,14457,118219,118220,117884,117879,267952,19721,117880
3,1,36135,5396,117961,118343,119993,118321,240983,290919,118322
4,1,42680,5905,117929,117930,119569,119323,123932,19793,119325


In [3]:
data.shape

(32769, 10)

In [4]:
# доля положительных примеров
data.ACTION.mean()

0.9421099209618847

In [5]:
# число значений у признаков
for col_name in data.columns:
    print(col_name, len(data[col_name].unique()))

ACTION 2
RESOURCE 7518
MGR_ID 4243
ROLE_ROLLUP_1 128
ROLE_ROLLUP_2 177
ROLE_DEPTNAME 449
ROLE_TITLE 343
ROLE_FAMILY_DESC 2358
ROLE_FAMILY 67
ROLE_CODE 343


In [6]:
from sklearn.cross_validation import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data.iloc[:, 1:], data.iloc[:, 0],
                                                    test_size=0.3, random_state=241)

#### 1. Реализуйте три функции расстояния на категориальных признаках, которые обсуждались на втором семинаре.

Проще всего будет определить метрики как [user-defined distance](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.DistanceMetric.html), после чего воспользоваться реализацией kNN из sklearn (в этом случае используйте функцию predict_proba). Можно реализовать метод k ближайших соседей и самостоятально — в этом случае учитите, что он должен возвращать оценку вероятности, то есть отношение объектов первого класса среди соседей к числу соседей).

Постарайтесь уделить особое внимание эффективности кода — при реализации метрик "в лоб" вы можете столкнуться с очень большим временем выполнения.

#### Подсчитайте для каждой из метрик качество на тестовой выборке `X_test` при числе соседей $k = 10$. Мера качества — AUC-ROC.

#### Какая функция расстояния оказалась лучшей?

In [7]:
from sklearn.neighbors import KNeighborsClassifier, DistanceMetric
from sklearn.metrics import roc_auc_score
from sklearn.cross_validation import KFold
from sklearn.preprocessing import StandardScaler

Реализации функций расстояния на категориальных признаках

In [8]:
class matchInd:
    def dist(self, x, y):
        return np.count_nonzero(x - y)
    
    
class matchIndCounts:
    def fit(self, X):
        self.nfeatures = X.shape[1]
        self.dicts = []
        for i in range(0, self.nfeatures):
            # создаем словарь (значение признака, log(f(x) + 1))
            values, counts = np.unique(X[:, i], return_counts=True)
            self.dicts.append(dict(np.hstack((values[:, np.newaxis], np.log(counts + 1)[:, np.newaxis]))))
        return None
    
    def dist(self, x, y):
        r = 0
        for i in np.where(x != y)[0]:
            r += self.dicts[i].get(x[i], 1) * self.dicts[i].get(y[i], 1)
        return r

    
class smoothMatch:
    def fit(self, X):
        self.nfeatures = X.shape[1]
        # словарь (значение признака x_j, кумулятивная сумма вариационного ряда из p^2(x_j))
        self.dicts = []
        for i in range(0, self.nfeatures):
            values, counts = np.unique(X[:, i], return_counts=True)
            N = counts.sum()
            if values.shape[0] == 1:
                raise ValueError('Все значения в признаке {} равны.'.format(i))
            ind = np.argsort(counts)
            counts = counts[ind]
            # вычисляем p^2(x_j)
            p2 = counts * (counts - 1) / (N * (N - 1))
            # сумма по всем q: p_j(q) <= p_j(x_j)
            p2sum = np.cumsum(p2)
            self.dicts.append(dict(np.hstack((values[ind, np.newaxis], p2sum[:, np.newaxis]))))
        return None
    
    def dist(self, x, y):
        r = np.sum(x != y)
        for i in np.where(x == y)[0]:
            r += self.dicts[i].get(x[i], 0)
        return r

In [9]:
mic = matchIndCounts()
mic.fit(X_train.values)
# micdist = DistanceMetric.get_metric(mic.dist)

sm = smoothMatch()
sm.fit(X_train.values)
# smdist = DistanceMetric.get_metric(sm.dist)

mi = matchInd()
# midist = DistanceMetric.get_metric(mi.dist)

In [13]:
import time

In [31]:
def dist(x, y):
    return np.count_nonzero(x - y)

In [47]:
start = time.time()
for i in range(0, 10):
    for j in range(0, X_train.shape[0]):
        sm.dist(X_test.values[i], X_train.values[j])
print((time.time() - start) / 10)

2.3242327690124513


In [ ]:
1.41 2.6 2.32

Сравнение метрик.

* midist --- Индикатор совпадения
* micdist --- Чем более частые значения оказались при несовпадении, тем больше расстояние
* smdist --- Сглаженный индикатор совпадения

In [12]:
clf = KNeighborsClassifier(n_neighbors=10, algorithm='brute', metric=mi.dist)
clf.fit(X_train.values, y_train.values)
answer = clf.predict_proba(X_test.values)
print('AUC_ROC = ', roc_auc_score(y_test.values, answer[:, 1]))

KeyboardInterrupt: 

In [ ]:
metrics = [mi.dist, mic.dist, sm.dist]
metric_names = ['midist', 'micdist', 'smdist']
for i in range(0, 3):
    print(metric_names[i])
    clf = KNeighborsClassifier(n_neighbors=10, algorithm='brute', metric=metrics[i], n_jobs=3)
    clf.fit(X_train.values, y_train.values)
    answer = clf.predict_proba(X_test.values)
    print('AUC_ROC = ', roc_auc_score(y_test.values, answer[:, 1]))

#### 2 (бонус). Подберите лучшее (на тестовой выборке) число соседей $k$ для каждой из функций расстояния. Какое наилучшее качество удалось получить?

Для подбора можно использовать любые средства из sklearn.

#### 3. Реализуйте счетчики (http://blogs.technet.com/b/machinelearning/archive/2015/02/17/big-learning-made-easy-with-counts.aspx), которые заменят категориальные признаки на вещественные.

А именно, каждый категориальный признак нужно заменить на три: 
1. Число `counts` объектов в обучающей выборке с таким же значением признака.
2. Число `successes` объектов первого класса ($y = 1$) в обучающей выборке с таким же значением признака.
3. Сглаженное отношение двух предыдущих величин: (`successes` + 1) / (`counts` + 2).

Поскольку признаки, содержащие информацию о целевой переменной, могут привести к переобучению, может оказаться полезным сделать *фолдинг*: разбить обучающую выборку на $n$ частей, и для $i$-й части считать `counts` и `successes` по всем остальным частям. Для тестовой выборки используются счетчики, посчитанные по всей обучающей выборке. Реализуйте и такой вариант. Можно использовать $n = 3$.

#### Посчитайте на тесте AUC-ROC метода $k$ ближайших соседей с евклидовой метрикой для выборки, где категориальные признаки заменены на счетчики. Сравните по AUC-ROC два варианта формирования выборки — с фолдингом и без. Не забудьте подобрать наилучшее число соседей $k$.

In [10]:
def make_counters(X, y, folding=True):
    n_samples = X.shape[0]
    n_features = X.shape[1]
    new_X = np.zeros((n_samples, 3 * n_features), dtype=float)
    if folding:
        kf = KFold(n_samples, shuffle=True)
    else:
        kf = [(np.arange(n_samples), np.arange(n_samples))]
    for train_ind, test_ind in kf:
        # train_ind - на основе чего строим счетчики
        for i in range(0, n_features):
            uniq_values, counts = np.unique(X[train_ind, i], return_counts=True)
            for j in range(0, uniq_values.shape[0]):
                ind = test_ind[np.where(X[test_ind, i] == uniq_values[j])[0]]
                new_X[ind, i] = counts[j] / train_ind.shape[0]  # учитываем размер train
                successes = np.sum(y[train_ind[np.where(X[train_ind, i] == uniq_values[j])[0]]])
                new_X[ind, n_features + i] = successes / train_ind.shape[0]
                new_X[ind, 2 * n_features + i] = (successes + 1) / (counts[j] + 2)
    return new_X

def make_counters_test(X_train, X_test, y_train):
    n_samples = X_test.shape[0]
    n_features = X_test.shape[1]
    new_X = np.zeros((n_samples, 3 * n_features), dtype=float)
    
    for i in range(0, n_features):
        uniq_values, counts = np.unique(X_train[:, i], return_counts=True)
        for j in range(0, uniq_values.shape[0]):
            ind = np.where(X_test[:, i] == uniq_values[j])[0]
            new_X[ind, i] = counts[j] / X_train.shape[0]
            successes = np.sum(y_train[np.where(X_train[:, i] == uniq_values[j])[0]])
            new_X[ind, n_features + i] = successes / X_train.shape[0]
            new_X[ind, 2 * n_features + i] = (successes + 1 / X_train.shape[0]) / (counts[j] + 2 / X_train.shape[0])
    return new_X    

In [ ]:
# X_train_nf = make_counters(X_train.values, y_train.values, folding=False)
X_train_f = make_counters(X_train.values, y_train.values, folding=True)
X_test_c = make_counters_test(X_train.values, X_test.values, y_train.values)

In [ ]:
for k in range(3, 20):
    clf = KNeighborsClassifier(n_neighbors=k, algorithm='brute')
    clf.fit(X_train_f, y_train.values)
    answer = clf.predict_proba(X_test_c)
    print('k = ', k, 'AUC-ROC = ', roc_auc_score(y_test.values, answer[:, ]))

In [48]:
?KN

#### 4. Добавьте в исходную выборку парные признаки — то есть для каждой пары $(f_i, f_j)$, $i < j$ исходных категориальных признаков добавьте новый категориальный признак $f_{ij}$, значение которого является конкатенацией значений $f_i$ и $f_j$ (желательно через какой-нибудь специальный символ во избежание коллизий). Посчитайте счетчики для этой выборки, найдите качество метода $k$ ближайших соседей с наилучшим $k$ (с фолдингом и без).

## Часть 3: Решающие деревья и леса

#### 1. Возьмите из предыдущей части выборку с парными признаками, преобразованную с помощью счетчиков без фолдинга. Настройте решающее дерево, подобрав оптимальные значения параметров `max_depth` и `min_samples_leaf`. Какой наилучший AUC-ROC на контроле удалось получить?

#### 2. Настройте случайный лес, подобрав такое число деревьев `n_estimators`, при котором ошибка выходит на асимптоту. Какое качество на тестовой выборке он дает?

#### 3. Возьмите выборку с парными признаками, для которой счетчики посчитаны с фолдингом. Обучите на ней случайный лес, подобрав число деревьев. Какое качество на тестовой выборке он дает? Чем вы можете объяснить изменение результата по сравнению с предыдущим пунктом?

Здесь вы можете поделиться своими мыслями о задании.

А здесь вставьте смешную картинку.

А здесь посоветуйте преподавателям хороший фильм или сериал.